# Libraries

In [21]:
!python3.11 -m pip install lightning
!python3.11 -m pip install torch

# Definitions

In [22]:
from torch import nn

class MLP(nn.Module):
    def __init__(self, width, depth):
      super().__init__()
      self.layers = nn.ModuleList()
      self.layers.append(nn.Linear(10, width))
      self.layers.append(nn.LeakyReLU())

      self.layers.extend([nn.Linear(width, width), nn.LeakyReLU()] * depth)

      self.layers.append(nn.Linear(width, 1))

    def forward(self, x):
      result = x
      for layer in self.layers:
          result = layer(result)
      return result

In [23]:
from torch.utils.data import Dataset
import pandas as pd

class OtwayData(Dataset):
  def __init__(self):
    """Loads dataset from file"""
    super().__init__()
    df = pd.read_csv("cleaned_data.csv").drop("Date", axis=1).astype('float32')
    # print(df)
    self.y = df["Close"].to_numpy()
    self.X = df.drop("Close", axis=1).to_numpy()
    self.len = self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]
  
  def __len__(self):
    return self.len

In [24]:
import lightning as L
import torch

class OtwayModel(L.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = MLP(width=128, depth=6)

  def loss_function(self, predicted_y, true_y):
    loss_fn = nn.MSELoss()
    return loss_fn(predicted_y.squeeze(), true_y)

  def training_step(self, batch, batch_idx):
    features, true_y = batch
    predicted_y = self.model(features)
    loss = self.loss_function(predicted_y, true_y)
    # self.log("train_loss", loss, on_epoch=True)
    return loss
  
  def validation_step(self, batch, batch_idx):
    features, true_y = batch
    predicted_y = self.model(features)
    loss = self.loss_function(predicted_y, true_y)
    self.log("val_loss", loss, on_epoch=True)
  
  def test_step(self, batch, batch_idx):
    features, true_y = batch
    predicted_y = self.model(features)
    loss = self.loss_function(predicted_y, true_y)
    self.log("test_loss", loss)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-6)
    return optimizer

In [27]:
from torch.utils.data import DataLoader, random_split

dataset = OtwayData()
train_set, val_set, test_set = random_split(dataset, [0.8, 0.1, 0.1])
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

# Training

In [32]:
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [29]:
model = OtwayModel()
trainer = L.Trainer(
  precision='32',
  max_epochs=-1,
  callbacks=[
    EarlyStopping(monitor="val_loss", mode="min", patience=5)
  ])
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type | Params
-------------------------------
0 | model | MLP  | 18.0 K
-------------------------------
18.0 K    Trainable params
0         Non-trainable params
18.0 K    Total params
0.072     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 592:  76%|███████▌  | 148/196 [00:00<00:00, 159.38it/s, v_num=2]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


# Evaluation

In [30]:
trainer.test(model=model, dataloaders=test_loader)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 25/25 [00:00<00:00, 234.11it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     5487.51416015625      │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 5487.51416015625}]

In [31]:
script = model.to_torchscript()

NotSupportedError: Compiled functions can't take variable number of arguments or use keyword-only arguments with defaults:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/lightning/pytorch/core/module.py", line 680
    @override
    def forward(self, *args: Any, **kwargs: Any) -> Any:
                                   ~~~~~~~ <--- HERE
        r"""Same as :meth:`torch.nn.Module.forward`.
    
